# Notebook de Teste da dblinea

In [3]:
!pip install dblinea

  Using cached dblinea-0.1.14-py3-none-any.whl (10 kB)
  Using cached psycopg2_binary-2.9.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)


In [4]:
from dblinea import DBBase

In [5]:
db = DBBase()
schema = "des_dr2"
table = "coadd_objects"

In [6]:
# Lista os bancos de dados disponiveis.
db.available_databases()

[{'config_name': 'gavo',
  'dbname': 'prod_gavo',
  'host': 'desdb4.linea.gov.br',
  'engine': 'postgresql_psycopg2'}]

In [7]:
# Fetchall SqlAlchemy
sql = "select coadd_object_id, ra, dec, mag_auto_g from {}.{} limit 5".format(schema, table)
rows = db.fetchall(sql)
print(rows)

[(884219024, 313.374483, -51.885184, 23.29636573791504), (884219207, 313.376736, -51.886442, 24.95577621459961), (884219493, 313.377553, -51.889549, 24.001747131347656), (884219570, 313.380125, -51.88971, 23.608625411987305), (884218699, 313.381525, -51.887442, 17.121112823486328)]


In [8]:
# Fetchall Dict
sql = "select coadd_object_id, ra, dec, mag_auto_g from {}.{} limit 5".format(schema, table)
rows = db.fetchall_dict(sql)
print(rows)

[{'coadd_object_id': 884219024, 'ra': 313.374483, 'dec': -51.885184, 'mag_auto_g': 23.29636573791504}, {'coadd_object_id': 884219207, 'ra': 313.376736, 'dec': -51.886442, 'mag_auto_g': 24.95577621459961}, {'coadd_object_id': 884219493, 'ra': 313.377553, 'dec': -51.889549, 'mag_auto_g': 24.001747131347656}, {'coadd_object_id': 884219570, 'ra': 313.380125, 'dec': -51.88971, 'mag_auto_g': 23.608625411987305}, {'coadd_object_id': 884218699, 'ra': 313.381525, 'dec': -51.887442, 'mag_auto_g': 17.121112823486328}]


In [9]:
# Fetchall Pandas Dataframe
import pandas as pd

sql = "select coadd_object_id, ra, dec, mag_auto_g from {}.{} limit 5".format(schema, table)
rows = db.fetchall_df(sql)
print(rows)

   coadd_object_id          ra        dec  mag_auto_g
0        884219024  313.374483 -51.885184   23.296366
1        884219207  313.376736 -51.886442   24.955776
2        884219493  313.377553 -51.889549   24.001747
3        884219570  313.380125 -51.889710   23.608625
4        884218699  313.381525 -51.887442   17.121113


In [10]:
# Fetchone SqlAlchemy
sql = "select coadd_object_id, ra, dec, mag_auto_g from {}.{} limit 5".format(schema, table)
row = db.fetchone(sql)
print(row)

(884219024, 313.374483, -51.885184, 23.29636573791504)


In [11]:
# Fetchone Dict
sql = "select coadd_object_id, ra, dec, mag_auto_g from {}.{} limit 5".format(schema, table)
row = db.fetchone(sql)
print(row)

(884219024, 313.374483, -51.885184, 23.29636573791504)


In [12]:
# Fetchscalar (retorna o valor da primeira linha na primeira coluna)
sql = "select coadd_object_id, ra, dec, mag_auto_g from {}.{} limit 5".format(schema, table)
a = db.fetch_scalar(sql)
print(a)

884219024


In [13]:
#List Table Columns 
columns = db.get_table_columns(table, schema)
print(columns[0:5])

['coadd_object_id', 'tilename', 'hpix_32', 'hpix_64', 'hpix_1024']


In [14]:
#Describe Table 
columns = db.describe_table(table, schema)
print(columns[0:5])

[{'name': 'coadd_object_id', 'type': BIGINT()}, {'name': 'tilename', 'type': TEXT()}, {'name': 'hpix_32', 'type': BIGINT()}, {'name': 'hpix_64', 'type': BIGINT()}, {'name': 'hpix_1024', 'type': BIGINT()}]


## Integração ScienceServer (DRI - TargetViewer)

Para a integração é necessário um token de autenticação, o usuario deve logar no science server, acessar o target viewer e no menu principal clicar em "API Token" e depois em "Generate Token"

Sobre o Token: 

- Apenas um token pode ser utilizado por vez.
- Toda vez que se clica em "Generate new token" o token antigo é removido.
- Copie o token em um lugar seguro.
- Após a janela "API Token" ser fechada o token não será mais visivel.

In [28]:
from dblinea import DBBase
from dblinea import ScienceServerApi

db = DBBase()
schema = "des_dr2"
table = "coadd_objects"

# Ler uma tabela do bando de dados
sql = "select coadd_object_id, ra, dec, mag_auto_g from {}.{} limit 5".format(schema, table)
# rows é uma lista de dicts
rows = db.fetchall_dict(sql)


token = '9b02a37f646592f67b2c5d011a3e28bf42d87245'

# Instancia da classe de intergração com Science Server
ss = ScienceServerApi(token)
# Importa os dados da tabela no Target Viewer
target_list = ss.target_list_from_list('My Target List', rows)
# print(target_list)
print(f" Target List ID: {target_list['id']} URL: {target_list['url']}")

 Target List ID: 153 URL: https://scienceserver.linea.gov.br/target/#cv/153


In [33]:
# Exemplo Importando um Pandas Dataframe direto para o target viewer
from dblinea import DBBase
from dblinea import ScienceServerApi

db = DBBase()
schema = "des_dr2"
table = "coadd_objects"

# Ler uma tabela do bando de dados
sql = "select coadd_object_id, ra, dec, mag_auto_g from {}.{} limit 5".format(schema, table)
# df é uma instancia de Pandas Dataframe
df = db.fetchall_df(sql)


token = '9b02a37f646592f67b2c5d011a3e28bf42d87245'

# Instancia da classe de intergração com Science Server
ss = ScienceServerApi(token)

# Importa os dados da tabela no Target Viewer
target_list = ss.target_list_from_list('My Target List', rows)

# print(target_list)
print(f" Target List ID: {target_list['id']} URL: {target_list['url']}")

 Target List ID: 155 URL: https://scienceserver.linea.gov.br/target/#cv/155


In [34]:
# Recuperar os metadados de uma target list pelo ID
from dblinea import ScienceServerApi

token = '9b02a37f646592f67b2c5d011a3e28bf42d87245'
ss = ScienceServerApi(token)
response = ss.get_catalog(155)
print(response)

{'id': 155, 'owner': 'gverde', 'date': '2022-06-30T18:52:16.992669Z', 'internal_name': 'my_target_list', 'display_name': 'My Target List', 'tbl_schema': 'dri_catalog', 'tbl_name': 'my_target_list', 'rows': 5, 'url': 'https://scienceserver.linea.gov.br/target/#cv/155'}


In [35]:
# Exemplo de como Remover uma Target List
from dblinea import ScienceServerApi

token = '9b02a37f646592f67b2c5d011a3e28bf42d87245'
ss = ScienceServerApi(token)
response = ss.remove_target_list(155)
print(response)

{'success': True, 'message': 'Target List successfully removed'}
